In [1]:
import sentence_transformers
from beir import util, LoggingHandler
from beir.retrieval import models as beir_models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
# from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from sentence_transformers import models, losses, datasets
from torch.utils.data import Dataset,DataLoader
import numpy as npR
from torch import nn, Tensor
from typing import Iterable, Dict
from sentence_transformers import SentenceTransformer
from transformers import BertConfig,AutoConfig, AutoTokenizer, AutoModelForCausalLM, PreTrainedModel,AutoModel
import logging
import os
import random
import torch
# from pretrain.arguments import ModelArguments
# from pretrain.enhancedDecoder import BertLayerForDecoder
from torch import nn
from transformers import BertForMaskedLM, AutoModelForMaskedLM
from transformers.modeling_outputs import MaskedLMOutput

from transformers.models.bert.modeling_bert import BertOnlyMLMHead

from MAELoss import MaskedAutoEncoderLoss
from MAEDataset import MaskedAutoEncoderDataset


import math
from typing import Optional, Tuple

import torch
import torch.utils.checkpoint
from torch import nn
from transformers.modeling_utils import (
    apply_chunking_to_forward,
    find_pruneable_heads_and_indices,
    prune_linear_layer,
)
from transformers.models.bert.modeling_bert import BertIntermediate, BertOutput, BertSelfOutput
from transformers.utils import (
    logging,
)
from transformers.models.auto.modeling_auto import MODEL_FOR_MASKED_LM_MAPPING


/Users/cgrdj/Documents/Code/sentence-transformers/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/cgrdj/Documents/Code/sentence-transformers/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/cgrdj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:

model_name = "huawei-noah/TinyBERT_General_4L_312D"
tokenizer = AutoTokenizer.from_pretrained(model_name,do_lower_case=False,clean_up_tokenization_spaces=False,clean_text=False)

In [3]:

HF_model=AutoModel.from_pretrained(model_name)

In [4]:
dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")
data_path = util.download_and_unzip(url, out_dir)
print("Dataset downloaded here: {}".format(data_path))
corpus, queries, qrels = GenericDataLoader(data_path).load(split="train") # or split = "train" or "dev"
unsupervised_train_data =  list(queries.values())#+[data['title']+' \n '+data['text'] for data in list(corpus.values())]
random.Random(0).shuffle(unsupervised_train_data)


Dataset downloaded here: /Users/cgrdj/Documents/Code/sentence-transformers/datasets/scifact


100%|██████████| 5183/5183 [00:00<00:00, 49403.40it/s]


In [5]:

word_embedding_model = models.Transformer(model_name, max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), "cls")
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [10]:
class MaskedAutoEncoderLoss(nn.Module):
    def __init__(self, model: SentenceTransformer):
        super(MaskedAutoEncoderLoss, self).__init__()
        self.encoder = model[0].auto_model  # This will be the final model used during the inference time.
        self.config = model[0].auto_model.config
        self.tokenizer = model.tokenizer
        self.MLM_head=BertOnlyMLMHead(self.config)
        self.cross_entropy = nn.CrossEntropyLoss()
        self.decoder = BertLayerForDecoder(self.config)
        self.decoder.apply(self.encoder._init_weights)
        
    def forward_exp(self, sentence_features: Iterable[Dict[str, Tensor]]):
        encoder_inputs,decoder_inputs, target_features= tuple(sentence_features)
        target_features[encoder_inputs.input_ids != self.tokenizer.mask_token_id] = -100
        last_hidden_states=self.encoder(**encoder_inputs)[0]
        lm_logits=self.MLM_head(last_hidden_states)
        loss = self.cross_entropy(lm_logits.view(-1, self.config.vocab_size),\
                                  target_features.view(-1))

        cls_hiddens = last_hidden_states[:, :1]
        embeddings=self.encoder.embeddings(input_ids=decoder_inputs)
        hiddens = torch.cat([cls_hiddens, embeddings[:, 1:]], dim=1)

        decoder_position_ids = self.embeddings.position_ids[:, :decoder_inputs.size(1)]
        decoder_position_embeddings = self.embeddings.position_embeddings(decoder_position_ids)  # B L D
        query = decoder_position_embeddings + cls_hiddens

        matrix_attention_mask = self.encoder.get_extended_attention_mask(
            decoder_attention_mask,
            decoder_attention_mask.shape,
            decoder_attention_mask.device
        )

        hiddens = self.c_head(query=query,
                              key=hiddens,
                              value=hiddens,
                              attention_mask=matrix_attention_mask)[0]
        
        return loss
    
loss=MaskedAutoEncoderLoss(model.cpu())


NameError: name 'BertLayerForDecoder' is not defined

In [47]:
bert_model=model[0].auto_model

In [55]:
bert_model.embeddings(
            encoder_inputs.input_ids,
            encoder_inputs.attention_mask,
            # labels=labels,
        )

tensor([[[-0.0394, -0.0294, -0.2240,  ...,  0.1635, -0.0193, -0.0673],
         [ 0.8163,  0.3312,  0.0357,  ..., -0.4310,  0.2611, -0.6336],
         [-0.3566,  0.3271,  0.1425,  ...,  0.0231,  0.4354, -0.4355],
         ...,
         [-0.4133,  0.0168,  0.2789,  ...,  0.5137,  0.7683,  0.4260],
         [-0.3264, -0.0387,  0.2875,  ...,  0.4052,  0.8615,  0.4296],
         [-0.4734,  0.0301,  0.2788,  ...,  0.4350,  0.6838,  0.3353]]],
       grad_fn=<NativeLayerNormBackward0>)

In [57]:
res=bert_model(
            encoder_inputs.input_ids,
            encoder_inputs.attention_mask,
            # labels=labels,
            output_hidden_states=True,
            return_dict=True
        )

In [72]:
bert_model.cls

AttributeError: 'BertModel' object has no attribute 'cls'

In [22]:
loss.forward_exp(tuple([inputs,labels]))

tensor(10.4593, grad_fn=<NllLossBackward0>)

In [23]:
labels_text = "Here is some text to encode"
encoder_text = "Here is some text to [MASK]"
decoder_text = "Here [MASK] [MASK] text [MASK] encode"
encoder_inputs = tokenizer(encoder_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
decoder_inputs = tokenizer(decoder_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

labels = tokenizer(labels_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").input_ids

# Vous devez définir les labels ici en fonction de votre stratégie de masquage
# Exemple simplifié où on suppose que le token masqué est à la position 5
labels[encoder_inputs.input_ids != tokenizer.mask_token_id] = -100

tensor([[-100, -100, -100, -100, -100, -100, 4372, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -

In [106]:
cross_entropy = nn.CrossEntropyLoss()
cross_entropy(
head(base_model(**inputs)[0]).view(-1, 30522),
labels.view(-1)
)


tensor(10.6928, grad_fn=<NllLossBackward0>)

In [11]:

class RetroMAEForPretraining(nn.Module):
    def __init__(
            self,
            bert: BertForMaskedLM,
    ):
        super(RetroMAEForPretraining, self).__init__()
        self.lm = bert

        self.decoder_embeddings = self.lm.bert.embeddings
        self.c_head = BertLayerForDecoder(bert.config)
        self.c_head.apply(self.lm._init_weights)

        # self.model_args = model_args

    def forward(self,
                encoder_input_ids, encoder_attention_mask, encoder_labels,
                decoder_input_ids, decoder_attention_mask, decoder_labels):
        
        lm_out: MaskedLMOutput = self.lm(
            encoder_input_ids, encoder_attention_mask,
            labels=encoder_labels,
            output_hidden_states=True,
            return_dict=True
        )
        print(lm_out)
        cls_hiddens = lm_out.hidden_states[-1][:, :1]  # B 1 D

        decoder_embedding_output = self.decoder_embeddings(input_ids=decoder_input_ids)
        hiddens = torch.cat([cls_hiddens, decoder_embedding_output[:, 1:]], dim=1)

        decoder_position_ids = self.lm.bert.embeddings.position_ids[:, :decoder_input_ids.size(1)]
        decoder_position_embeddings = self.lm.bert.embeddings.position_embeddings(decoder_position_ids)  # B L D
        query = decoder_position_embeddings + cls_hiddens

        matrix_attention_mask = self.lm.get_extended_attention_mask(
            decoder_attention_mask,
            decoder_attention_mask.shape,
            decoder_attention_mask.device
        )

        hiddens = self.c_head(query=query,
                              key=hiddens,
                              value=hiddens,
                              attention_mask=matrix_attention_mask)[0]
        pred_scores, loss = self.mlm_loss(hiddens, decoder_labels)

        return (loss + lm_out.loss,)

    def mlm_loss(self, hiddens, labels):
        pred_scores = self.lm.cls(hiddens)
        masked_lm_loss = self.cross_entropy(
            pred_scores.view(-1, self.lm.config.vocab_size),
            labels.view(-1)
        )
        return pred_scores, masked_lm_loss

    def save_pretrained(self, output_dir: str):
        self.lm.save_pretrained(output_dir)

    @classmethod
    def from_pretrained(
            cls,
            #   model_args: ModelArguments,
            *args, **kwargs
    ):
        hf_model = AutoModelForMaskedLM.from_pretrained(*args, **kwargs)
        model = cls(hf_model,
                    #  model_args
                     )
        return model

In [12]:
'''
The codes are modified based on huggingface transformers library.
'''


logger = logging.get_logger(__name__)


class BertSelfAttention(nn.Module):
    def __init__(self, config, position_embedding_type=None):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention "
                f"heads ({config.num_attention_heads})"
            )

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
        self.position_embedding_type = position_embedding_type or getattr(
            config, "position_embedding_type", "absolute"
        )
        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            self.max_position_embeddings = config.max_position_embeddings
            self.distance_embedding = nn.Embedding(2 * config.max_position_embeddings - 1, self.attention_head_size)

        self.is_decoder = config.is_decoder

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
            self,
            query,
            key,
            value,
            attention_mask: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            encoder_hidden_states: Optional[torch.FloatTensor] = None,
            encoder_attention_mask: Optional[torch.FloatTensor] = None,
            past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        mixed_query_layer = self.query(query)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(key))
            value_layer = self.transpose_for_scores(self.value(value))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(key))
            value_layer = self.transpose_for_scores(self.value(value))

        query_layer = self.transpose_for_scores(mixed_query_layer)

        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            seq_length = query.size()[1]
            position_ids_l = torch.arange(seq_length, dtype=torch.long, device=query.device).view(-1, 1)
            position_ids_r = torch.arange(seq_length, dtype=torch.long, device=query.device).view(1, -1)
            distance = position_ids_l - position_ids_r
            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in BertModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.functional.softmax(attention_scores, dim=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs


class BertAttention(nn.Module):
    def __init__(self, config, position_embedding_type=None):
        super().__init__()
        self.self = BertSelfAttention(config, position_embedding_type=position_embedding_type)
        self.output = BertSelfOutput(config)
        self.pruned_heads = set()

    def prune_heads(self, heads):
        if len(heads) == 0:
            return
        heads, index = find_pruneable_heads_and_indices(
            heads, self.self.num_attention_heads, self.self.attention_head_size, self.pruned_heads
        )

        # Prune linear layers
        self.self.query = prune_linear_layer(self.self.query, index)
        self.self.key = prune_linear_layer(self.self.key, index)
        self.self.value = prune_linear_layer(self.self.value, index)
        self.output.dense = prune_linear_layer(self.output.dense, index, dim=1)

        # Update hyper params and store pruned heads
        self.self.num_attention_heads = self.self.num_attention_heads - len(heads)
        self.self.all_head_size = self.self.attention_head_size * self.self.num_attention_heads
        self.pruned_heads = self.pruned_heads.union(heads)

    def forward(
            self,
            query: torch.Tensor,
            key: torch.Tensor,
            value: torch.Tensor,
            attention_mask: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            encoder_hidden_states: Optional[torch.FloatTensor] = None,
            encoder_attention_mask: Optional[torch.FloatTensor] = None,
            past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        self_outputs = self.self(
            query, key, value,
            attention_mask,
            head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
        )
        attention_output = self.output(self_outputs[0], query)
        outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
        return outputs


class BertLayerForDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.chunk_size_feed_forward = config.chunk_size_feed_forward
        self.seq_len_dim = 1
        self.attention = BertAttention(config)
        self.is_decoder = config.is_decoder
        self.add_cross_attention = config.add_cross_attention
        if self.add_cross_attention:
            if not self.is_decoder:
                raise ValueError(f"{self} should be used as a decoder model if cross attention is added")
            self.crossattention = BertAttention(config, position_embedding_type="absolute")
        self.intermediate = BertIntermediate(config)
        self.output = BertOutput(config)

    def forward(
            self,
            query: torch.Tensor,
            key: torch.Tensor,
            value: torch.Tensor,
            attention_mask: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            encoder_hidden_states: Optional[torch.FloatTensor] = None,
            encoder_attention_mask: Optional[torch.FloatTensor] = None,
            past_key_value: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            output_attentions: Optional[bool] = False,
    ) -> Tuple[torch.Tensor]:
        # decoder uni-directional self-attention cached key/values tuple is at positions 1,2
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        self_attention_outputs = self.attention(
            query, key, value,
            attention_mask,
            head_mask,
            output_attentions=output_attentions,
            past_key_value=self_attn_past_key_value,
        )
        attention_output = self_attention_outputs[0]

        # if decoder, the last output is tuple of self-attn cache
        if self.is_decoder:
            outputs = self_attention_outputs[1:-1]
            present_key_value = self_attention_outputs[-1]
        else:
            outputs = self_attention_outputs[1:]  # add self attentions if we output attention weights

        cross_attn_present_key_value = None
        if self.is_decoder and encoder_hidden_states is not None:
            if not hasattr(self, "crossattention"):
                raise ValueError(
                    f"If `encoder_hidden_states` are passed, {self} has to be instantiated with cross-attention layers by setting `config.add_cross_attention=True`"
                )

            # cross_attn cached key/values tuple is at positions 3,4 of past_key_value tuple
            cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
            cross_attention_outputs = self.crossattention(
                attention_output,
                attention_mask,
                head_mask,
                encoder_hidden_states,
                encoder_attention_mask,
                cross_attn_past_key_value,
                output_attentions,
            )
            attention_output = cross_attention_outputs[0]
            outputs = outputs + cross_attention_outputs[1:-1]  # add cross attentions if we output attention weights

            # add cross-attn cache to positions 3,4 of present_key_value tuple
            cross_attn_present_key_value = cross_attention_outputs[-1]
            present_key_value = present_key_value + cross_attn_present_key_value

        layer_output = apply_chunking_to_forward(
            self.feed_forward_chunk, self.chunk_size_feed_forward, self.seq_len_dim, attention_output
        )
        outputs = (layer_output,) + outputs

        # if decoder, return the attn key/values as the last output
        if self.is_decoder:
            outputs = outputs + (present_key_value,)

        return outputs

    def feed_forward_chunk(self, attention_output):
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

AttributeError: module 'logging' has no attribute 'get_logger'

In [207]:
train_dataset=MaskedAutoEncoderDataset(unsupervised_train_data,AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D'))
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True)
train_loss = MaskedAutoEncoderLoss(model, decoder_name_or_path=model_name)


Some weights of BertLMHeadModel were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.self.key.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [189]:
modelMae=RetroMAEForPretraining.from_pretrained( pretrained_model_name_or_path="huawei-noah/TinyBERT_General_4L_312D")

In [219]:
train_dataset[2].texts

['Ambulatory', 'blood', 'pressure', 'monitoring', 'is', 'inaccurate', 'at', 'diagnosing', 'hypertension.']
[[2572, 28507, 7062], [2668], [3778], [8822], [2003], [24949], [2012], [22939, 26745, 7741], [23760, 29048, 1012]]
[[2572, 28507, 7062], [2668], [3778], [8822], [2003], [24949], [2012], [22939, 26745, 103], [23760, 29048, 1012]]
['Ambulatory', 'blood', 'pressure', 'monitoring', 'is', 'inaccurate', 'at', 'diagnosing', 'hypertension.']
[[2572, 28507, 7062], [2668], [3778], [8822], [2003], [24949], [2012], [22939, 26745, 7741], [23760, 29048, 1012]]
[[103, 28507, 103], [103], [103], [8822], [2003], [24949], [2012], [22939, 26745, 7741], [23760, 29048, 103]]


['ambulatory blood pressure monitoring is inaccurate at diagno[MASK] hypertension.',
 '[MASK]bula[MASK] [MASK] [MASK] monitoring is inaccurate at diagnosing hypertension[MASK]',
 'Ambulatory blood pressure monitoring is inaccurate at diagnosing hypertension.']

In [ ]:
encoder_attention_mask[0]


In [13]:
modelMae(tokenizer(['ceci est un test'],return_tensors='pt'))

TypeError: RetroMAEForPretraining.forward() missing 5 required positional arguments: 'encoder_attention_mask', 'encoder_labels', 'decoder_input_ids', 'decoder_attention_mask', and 'decoder_labels'

In [35]:

decoder_config = AutoConfig.from_pretrained('gpt2')
decoder_config.is_decoder = True
decoder_config.add_cross_attention = True
decoder_config.num_hidden_layers=1
kwargs_decoder = {"config": decoder_config}
decoder = AutoModelForCausalLM.from_pretrained('gpt2', **kwargs_decoder)
tokenizer=AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of the model checkpoint at gpt2 were not used when initializing GPT2LMHeadModel: ['h.3.ln_1.weight', 'h.3.ln_1.bias', 'h.11.mlp.c_proj.weight', 'h.6.mlp.c_proj.weight', 'h.11.attn.c_proj.weight', 'h.2.attn.c_attn.bias', 'h.8.ln_2.bias', 'h.1.mlp.c_fc.weight', 'h.3.mlp.c_fc.bias', 'h.5.attn.c_attn.bias', 'h.6.attn.c_proj.bias', 'h.1.attn.c_attn.bias', 'h.10.attn.c_attn.bias', 'h.5.mlp.c_fc.bias', 'h.1.mlp.c_proj.weight', 'h.7.mlp.c_proj.weight', 'h.10.attn.c_proj.bias', 'h.8.mlp.c_proj.bias', 'h.4.attn.c_proj.bias', 'h.3.mlp.c_proj.bias', 'h.8.attn.c_attn.weight', 'h.7.attn.c_attn.weight', 'h.8.attn.bias', 'h.2.ln_2.bias', 'h.11.ln_1.weight', 'h.6.attn.c_attn.weight', 'h.6.mlp.

In [186]:
train_dataset=datasets.DenoisingAutoEncoderDataset(unsupervised_train_data)#,tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True)
train_loss = MaskedAutoEncoderLoss(model)


Some weights of BertLMHeadModel were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [187]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    # evaluator=dev_evaluator,
    epochs=1,
    # evaluation_steps=evaluation_steps,
    # output_path=model_save_path,
    weight_decay=0,
    warmup_steps=100,
    optimizer_params={"lr": 3e-5},
    use_amp=True,  # Set to True, if your GPU supports FP16 cores
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

tensor([[18847, 20464, 16026, 27781, 14126,  1997,  1050,  1011, 28353,  5886,
          2378, 16171,  3459,  8156,  5012,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [ 2512,  2378, 12044,  3512,  3893,  3778, 19536,  2003,  2025, 16014,
          3512,  1997, 11325, 16464,  4945,  2044,  5024,  5812, 22291,  3370,
          1012,   102,     0,     0,     0,     0],
        [ 3806, 12412, 13697,  3351,  2003,  2019,  4621,  3949,  2005, 11325,
         11498, 16211,  2102, 16149,  1012,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [13822,  1997,  1042, 23518,  5648,  1006,  6904,  1007,  1998, 17663,
          1038,  2575,  1006,  1058,  2497,  2575,  1007, 13416,  3798,  1997,
         24004,  5666,  8602,  2063,  1012,   102]]) tensor([[  101, 18847, 20464, 16026, 27781, 14126,  1997,  1050,  1011, 28353,
          5886,  2378, 16171,  3459,  8156,  5012,  1012,   102,     0,     0,
 

tensor([[19395, 19137,  5302, 15069,  3111,  2031,  1037,  2152,  3014,  1997,
          3526,  1998,  8153,  3563, 19314,  1012,   102,     0,     0,     0,
             0,     0],
        [ 2012,  2546,  2549,  2003,  1037,  2236,  2203,  7361,  8523,  7712,
          2128,  4588, 25100,  6911, 12115,  1012,   102,     0,     0,     0,
             0,     0],
        [18672,  8844, 26450, 11110,  2024,  2062,  4621,  2084,  2236,  6912,
          7242,  1999,  8161,  3255,  1998,  9229,  3853,  1997,  1996,  3244,
          1012,   102],
        [ 6335,  1011,  1020, 14828,  3248,  1037,  2350,  2535,  1999,  2012,
          5886,  2891, 14321, 21709,  2594, 22935,  4295,  1012,   102,     0,
             0,     0]]) tensor([[  101, 19395, 19137,  5302, 15069,  3111,  2031,  1037,  2152,  3014,
          1997,  3526,  1998,  8153,  3563, 19314,  1012,   102,     0,     0,
             0,     0],
        [  101,  2012,  2546,  2549,  2003,  1037,  2236,  2203,  7361,  8523,
          

tensor([[ 3465, 12353,  9312,  2015,  2241,  2006, 13675,  6593,  2951, 14125,
          8339,  7597,  2005,  5022,  1999,  5025,  6612,  3218,  1012,   102,
             0,     0,     0,     0,     0],
        [ 2512, 10536,  4842, 25808,  3512,  2111,  2040,  2024,  4583,  2086,
          2214,  2031,  1037,  3938,  1003,  3382,  1997,  4975, 23760, 29048,
          2076,  2037,  6480,  1012,   102],
        [ 1054,  1011,  5250, 15775,  4842, 21821,  2024,  2179,  1999, 15420,
          2007,  2037,  1054,  1011,  5250,  8031,  5826,  1012,   102,     0,
             0,     0,     0,     0,     0],
        [ 2396,  2038,  2053,  3466,  2006,  1996,  1999, 25969,  3512,  2791,
          1997,  9820,  1011,  3893,  2111,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0]]) tensor([[  101,  3465, 12353,  9312,  2015,  2241,  2006, 13675,  6593,  2951,
         14125,  8339,  7597,  2005,  5022,  1999,  5025,  6612,  3218,  1012,
           102,     0,     0,

tensor([[ 3078, 28711, 22330, 23479, 11326,  2007,  6522,  2615, 10788,  2038,
          3020, 20134, 14639,  2000, 11487,  5729, 28711, 26721, 13699,  8939,
         24587,  9253, 24759, 15396,  2084,  7511, 22330, 23479,  1012,   102],
        [ 3690,  2361,  2487,  1055, 16275,  2015,  2024,  4958, 11921,  4588,
          2000,  1044,  2721,  2035, 26741,  2005,  2019,  4801, 10483,  2075,
         11867, 15422,  8516, 13706,  1012,   102,     0,     0,     0,     0],
        [ 1999,  5022,  2007, 27480,  2540, 18419,  1010,  1156,  1011,  3796,
          2545,  6022,  5335,  6612,  2540,  4945, 13105,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 5423,  7361,  4747,  7274,  6305,  7507, 15637,  2015,  2031,  2019,
         21733,  2981,  3466,  2006, 14234,  8803,  3853,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]) tensor([[  101,  3078, 28711, 22330, 23479, 113

...

tensor([[ 1996, 16192,  2565, 16691,  2053,  3278,  4254,  2006, 16012, 15869,
         13105,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [26402,  2075, 18423,  1011,  1020,  1011,  6887,  2891, 17585,  2139,
         10536, 22196, 28835, 24840,  5423, 23924, 19009,  2011, 26709,  6593,
         17441,  1048,  2243,  2497,  2487,  1011, 23713,  2243, 14828,  1012,
           102],
        [ 1051,  4017, 23691, 24873,  6632,  2278,  5022,  2089,  2031,  1051,
          4017,  3563, 20187,  4442,  1999,  2037,  2235,  6812,  2884, 14163,
         13186,  2050,  1012,   102,     0,     0,     0,     0,     0,     0,
             0],
        [ 2273,  2024,  2062, 18002,  2000,  2331,  2349,  2000, 18583,  2043,
          4102,  2000,  2308,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
 

tensor([[28157, 14828,  2013,  1996,  8040,  2546,  2615,  2003, 26986,  2011,
          2026,  2094,  2620,  2620,  1013,  3729, 12740,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [18847, 20464, 16026, 27781, 14126,  1997,  1050,  1011, 28353,  5886,
          2378, 16171, 18804,  9153,  6190,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [16480,  4244, 27833, 10578, 19653,  2015,  1047, 10270,  2549,  3670,
          1999,  5443, 12458,  2015,  1010,  4525,  1999,  1996,  3670,  1997,
          4013,  1011, 20187, 22330, 18715, 10586,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [ 6970,  7512,  2239, 25194,  9165,  1006,  2003,  5620,  1007, 20868,
          2546,  2487,  1010, 208

tensor([[ 6583,  2278, 26402,  2015,  1996,  4245,  1997, 17076, 25185,  6132,
          2378,  1011, 16401,  9007,  1012,   102,     0],
        [ 3768,  1997,  1042, 25708, 17465,  1999, 12328,  7457,  2166,  5987,
         11656,  1012,   102,     0,     0,     0,     0],
        [ 9680, 24079, 15459, 14350, 12520,  1999,  5909, 10029,  1012,   102,
             0,     0,     0,     0,     0,     0,     0],
        [ 1996,  3891,  1997,  2931,  5895,  7386,  2075,  3209,  2003,  2702,
          2335,  2008,  1997,  3287,  5895,  1012,   102]]) tensor([[  101,  6583,  2278, 26402,  2015,  1996,  4245,  1997, 17076, 25185,
          6132,  2378,  1011, 16401,  9007,  1012,   102],
        [  101,  3768,  1997,  1042, 25708, 17465,  1999, 12328,  7457,  2166,
          5987, 11656,  1012,   102,     0,     0,     0],
        [  101,  9680, 24079, 15459, 14350, 12520,  1999,  5909, 10029,  1012,
           102,     0,     0,     0,     0,     0,     0],
        [  101,  1996,  3891,  19

Epoch:   0%|          | 0/1 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [94]:
next(iter(MaskedAutoEncoderDataset(["asdasdf adsf sda fads 'asd gasdg aasdfasdf /dsaf sadfsd,dasfadsf. fadsf.fads,fadsfa"],tokenizer))).texts

['[MASK]dasdf ads[MASK] sda [MASK]ds [MASK]asd gasdg aasdfa[MASK]f /dsaf sadfsd,dasfa[MASK]f. [MASK]ds[MASK].fads,fadsfa',
 '[MASK]dasdf [MASK][MASK] sda fads [MASK]asd gas[MASK]g [MASK]sd[MASK][MASK][MASK] [MASK][MASK]af [MASK][MASK][MASK][MASK]dasfadsf[MASK] [MASK][MASK][MASK].fads[MASK]fads[MASK]',
 "asdasdf adsf sda fads 'asd gasdg aasdfasdf /dsaf sadfsd,dasfadsf. fadsf.fads,fadsfa"]

In [77]:
sentence="asdasdf adsf sda fads 'asd gasdg aasdfasdf /dsaf sadfsd,dasfadsf. fadsf.fads,fadsfa"
tokenizer.decode(tokenizer.encode_plus(sentence,return_attention_mask=False,return_token_type_ids=False,add_special_tokens=False)['input_ids'])



"asdasdf adsf sda fads ' asd gasdg aasdfasdf / dsaf sadfsd, dasfadsf. fadsf. fads, fadsfa"

In [5]:
tokenizer.decode(tokenizer.encode('as [MASK]df',add_special_tokens=False))



'as [MASK] df'

In [6]:

sentences=["asdasdf adsf sda fads 'asd gasdg aasdfasdf /dsaf sadfsd,dasfadsf. fadsf.fads,fadsfa"]*10000
masked_sentences=[]
for sentence in sentences:
    words= word_tokenize(sentence)
    # Apply the masking logic to each word and rejoin the sentence
    splitted_tokens = tokenizer.batch_encode_plus(words,return_attention_mask=False,return_token_type_ids=False,add_special_tokens=False)['input_ids']
    masked_sentence=' '.join([tokenizer.decode([ mask_id if np.random.rand() < mask_probability else tok_id for tok_id in word]).replace(" ",'') for word in splitted_tokens])
    masked_sentences.append(masked_sentence)



# masked_sentence = ' '.join([mask_token if np.random.rand() < mask_probability else word for word in words])
        

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/cgrdj/nltk_data'
    - '/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/nltk_data'
    - '/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/share/nltk_data'
    - '/home/cgrdj/Documents/code/repos/sentence-transformers/.conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [369]:
masked_sentence

[6904, 5104, 2546, 1012, 6904, 5104]

In [416]:
remove_spaces_table=str.maketrans('', '', ' ')


In [431]:
%%timeit 
tokenizer.decode([ mask_id if np.random.rand() < mask_probability else tok_id for tok_id in splitted_tokens[12]]).replace(" ",'')


53.5 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [429]:
%%timeit -n 100000
tokenizer.decode([ mask_id if np.random.rand() < mask_probability else tok_id for tok_id in splitted_tokens[12]]).translate(remove_spaces_table)


59.8 µs ± 4.79 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [385]:
tokenizer('f')

{'input_ids': [101, 1042, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [227]:
tokenizer.convert_tokens_to_string(tokenizer(sentence)['input_ids'])



TypeError: argument 'tokens': 'int' object cannot be converted to 'PyString'

In [212]:
mask_probability=0.15
mask_token = tokenizer.mask_token  # Get the mask token
tokens = tokenizer.tokenize(sentence)
# Decide randomly which tokens to mask
masked_indices = np.random.rand(len(tokens)) < mask_probability
# Replace selected tokens with the mask token
masked_tokens = [mask_token if mask else token for token, mask in zip(tokens, masked_indices)]
# Convert the list of tokens back to a string
masked_sentence = tokenizer.convert_tokens_to_string(masked_tokens)
# Add the masked sentence to the list
# masked_sentences.append(masked_sentence)
masked_sentence

"asdas [MASK] adsf sda fads ' asd gas [MASK]g aasdfasdf / dsaf sad [MASK]d, [MASK]fadsf. fadsf. fa [MASK], fadsfa"

In [208]:
masked_sentence

"as [MASK]df adsf sd [MASK] fads ' asd gas [MASK]g aa [MASK]fasdf [MASK] [MASK] [MASK] sadfsd, dasfadsf. [MASK] [MASK] [MASK]. fads [MASK] fa [MASK]fa"

In [203]:

word_tokenize(sentence)




['asdasdf',
 'adsf',
 'sda',
 'fads',
 "'asd",
 'gasdg',
 'aasdfasdf',
 '/dsaf',
 'sadfsd',
 ',',
 'dasfadsf',
 '.',
 'fadsf.fads',
 ',',
 'fadsfa']

# Data

In [192]:
dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")
data_path = util.download_and_unzip(url, out_dir)
print("Dataset downloaded here: {}".format(data_path))

Dataset downloaded here: /Users/cgrdj/Documents/Code/sentence-transformers/datasets/scifact


In [193]:
corpus, queries, qrels = GenericDataLoader(data_path).load(split="train") # or split = "train" or "dev"
unsupervised_train_data =  list(queries.values())#+[data['title']+' \n '+data['text'] for data in list(corpus.values())]
random.Random(0).shuffle(unsupervised_train_data)


2024-03-16 00:49:24 - Loading Corpus...


100%|██████████| 5183/5183 [00:00<00:00, 11485.08it/s]

2024-03-16 00:49:24 - Loaded 5183 TRAIN Documents.
2024-03-16 00:49:24 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 ver

In [194]:
data_path = "datasets/scifact"
test_corpus, test_queries, test_qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

2024-03-16 00:49:25 - Loading Corpus...


  0%|          | 0/5183 [00:00<?, ?it/s]

100%|██████████| 5183/5183 [00:00<00:00, 16216.28it/s]


2024-03-16 00:49:26 - Loaded 5183 TEST Documents.
2024-03-16 00:49:26 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers